### This will be in the prepare module
    _Check for and remove duplicates by customer_id. - None
    
    _Remove redundant columns: payment_type_id, internet_service_type_id, contract_type_id, & customer_id. - Completed
        -Used the following to drop redundant columns
        -df = df.drop(['payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id'], axis = 1)
        -df.head(2)
    
    _Encode payment_type(Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic)), internet_service_type(DSL, Fiber, None), contract_type(Month-to-month, One year, Two year), gender(male/female), partner(yes/no), multiple_line(yes/no), device_protection(yes/no), tech_support(yes/no), streaming_tv(yes/no), streaming_movies(yes/no), paperless_billing, & churn (yes/no).
    - gender is male(1) or not male(0)
    - yes(1) no(0)
    - payment, contract and service types (True = 1) (Fales = 0)
    
    _Check for nulls and fill (Fill in total_charges' blanks with 0). - Completed
        -#find empty values. Tried isna, isnull, notna, notnull but did not shown any matches.
            -df.eq(' ').sum()
        -Used .replace to empty values with a 0 in total charges.
            -df = df.replace({'total_charges': ' '}, 0)
            -df.head()

    _Convert total_charges to float64. - Completed
        -#convert 'total_charges' to float and validate change.
        - df['total_charges'] = df['total_charges'].astype(float)
        - df.dtypes
    
    _Replace 'No phone service' & 'No internet service' with 'No'. - Completed
        -df.replace(to_replace = 'No internet service', value = 'No')
        -df.replace(to_replace = 'No phone service', value = 'No')

In [ ]:
#libraries
import get_db
import explore
import prepare
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

import graphviz
from graphviz import Graph

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

#sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Plan

    Goal: Identify feature(s) that cause the greatest churn.
    
    Churn rate is 26.5% (1869) of 7043 customers.
    
    Taking you through the data pipeline I will begin with acquiring the telcor data from the get_db.py module, prepare it using the prepare.py module, split the data and on through exploration. Using explore.py module I identified a set of features used to develop my initial hypothesis. Once cleaned I ran it through multivariate to compare features to help with testing the listed inital hypothesis. Each exploration will include takeaways that have led to preparing, cleaning ant testing of the data resulting in rejecting or failing to reject the final hypothesis.
    
    Features within my hypothesis have been split, tested and modeled in order to provide a recommendation on the final hypothesis in order to reduce churn rates based rejected or failed to reject of the hypothesis. 
    
    I have also identified unknown variables that at a later time can be researched, developed and tested using this model to predict future churn.

    My initial hypothesis are listed below:
    
    1. churn is dependent of whether or not customers are on fiber. (chi2)
        - Null: churn is independent of whether or not a customer are on fiber.
        - Alternate: churn is dependent of whether or not customers are on fiber.
            NOTES from univariate TAKEAWAYS:
                *****AND what internet services fiber customers have AND do they affect churn?
                *****What are the monthly costs?
                
    2. tenure and monthly charges are linearly correlated (pearsonr)
        - Null: tenure and monthly charges are not linearly correlated
        - Alt: tenure and monthly charges are linearly correlated
                
    3. churn is dependent younger customers.(chi2)
        - Null: churn is independent of younger customers.
        - Alternate: churn is dependent younger customers.
        
    4. Are monthly charges different for older people(ttest)
        N: The average monthly charges for older customers is no different than the population (monthly_charges_mean)
        A: The average monthly charges for older customers are different than rest of the population.
            
    5. do older people churn more than younger people(ttest)
        -avg churn for older customers is no different than the population (churn)
        -avg churn for older customers is different thatn the pop
    
    6. churn dependent on autopayment.(chi2)
        - Null: churn is independent of whether or not customes are enrolle in autopayment plan.
        - Alternate: churn is dependent of whether or not customes are enrolle in autopayment plan.
            
    7. churn dependent on phone_service (951)(chi2)
            - Null: churn is independent on phone services
            - Alternate: churn is dependent on phone services
            
    FEATURES:
    
        *** fiber (YES), tenure (1 month), single (NO), senior citizen (NO), m2m (YES), autopayments (NO), phone_service (YES)

    FINAL HYPOTHESIS (reject or fail to reject):

        *** Customers who are young, not single, on fiber, has phone service, on m2m, and not enrolled in a autopayment plan are more likely to churn than those customers that do not fall in this category.
        

    Exploration:
        1. Univariate
        2. Bivariate
        3. Multivariate

    Modeling:
        1. Decision Tree
        2. Random Forrest
        3. KNN
        
    Conclusion
        In conclusion, we ran through the data pipeline, identified our initial hypothesis, split, tested and model our features that resulted in the following recommendations.
        1.
        2.
        3.


# *Working code for project*

# Acquire

In [ ]:
df = get_db.get_telco_data()
df.head()

#### Initial visualization

##### TAKEAWAYS 

    -Related services:
        *multiple_services relates to phone_services
        *online_security, online_backup, device_protection, tech_support, streaming_tv, and streaming_movies related to internet services. 

# Clean/Prepare

    - The prepare file is broke up by sections to identify at which stage of the pipeline that I decided to edit, alter, modify or delete colums, column names, data types, etc...and are identifed by notes within the code.

    * Initial clean up was done just after initial visualization of raw data.
    * all other decisions to modify the dataframe are annotated within the prepare.py module.

In [ ]:
#added above to the prepare module.
df = prepare.clean_data(df)
df.head()

# Split your data into train, validate, and test samples.

    The resulting dataframes should be 3 samples:
    
        1. a dataframe for training the algorithms
            * The train dataset is for training our models. We also perform our exploratory data analysis on train.
            
        2. a dataframe for validating the models developed on unseen data
            *First, it is an "out of sample" dataset so that we can evaluate our models on unseen data to measure how well the model generalizes.
            *Second, the validate set allows us to fine tune our hyperparameters.
            
        3. a dataframe for testing the best performing model to ensure the model is able to be generalized on a final set of unseen data not 'overfitting' train, a validate and a test, roughly a 70%:20%:10% split
            *final out of sample dataset used to evaluate how well the models tuned on validate generalize on unseen data.

In [ ]:
#Split the dataframe into three dataframes (train, validate, and test)
train, validate, test = prepare.train_validate_test_split(df, 'churned')

# Exploratory

    1. Discover features, using  that have the largest impact on the target variable, i.e. provide the most information gain, drive the outcome. To discover patterns, to spot anomalies, to test hypotheses and to check assumptions
        *The dataframe resulting from these functions should be one that is pre-processed, i.e. ready to be used in modeling.
        *attributes are reduced to features, features are in a numeric form, there are no missing values, and continuous and/or ordered values are scaled to be unitless.
        
    2. Hypothesize: Form and document your initial hypotheses about how the predictors (independent variables, features, or attributes) interact with the target (y-value or dependent variable).
    
    3. Use visualization techniques to identify drivers and follow up with a statistical test, do so.
    
    4. Test Hypothesis (t-tests, correlation, chi-square, e.g.)

#### Univariate Stats

    1. Descriptive stats, frequencies, histograms.    
        * Explore the target variable.
        * Explore the categorical/qualitative variables.
        * Explore the continuous/quantitative variables.

In [ ]:
train.shape

In [ ]:
#list all categorical variables and quatintative variables for exploration within the train df.
cat_vars = ['fiber', 'm2m', 'is_senior', 'churned', 'not_single', 'phone_service',
           'online_security', 'online_backup', 'device_protection', 'tech_support', 'autopayment']
quant_vars = ['tenure', 'monthly_charges']
#variable histograms
explore.explore_univariate(train, cat_vars, quant_vars)

##### TAKEAWAYS
    8/28/2021
    Of the TRAIN data...
    - 44.43% (1752) of customers are on fiber
    - 54.78% (2160) of customers are m2m 
    - 82.7% (3261) of customers are not senior
    - 26.53% (1046) of customers churned
    - 46.79% (1845) of customers are not single
    - is_male is an even distribution. DROPPED this column.
    - Customers with a 'partner' similarly divided (47.6 & 52.6 respectively)
    - Customers with 'dependents' account for 70.1%
    - Customers with 'phone_service_ account' for 91%
    - autopayment accounts for 56.81% (2240) of customers 43.19% (1703) are not on autopayment.
    - phone and internet services will be used at a later time.
    - average tenure 32.32%
    - average monthly_charges 65.15%
    
    -Initial assessment during prepare stage of the pipeline.
        - m2m - 2160 or 55%, 1 year - 811 or 21%, 2 year - 972 or 24%
            *change contract type to m2m (yes or no) and drop one and two year contracts.
            *assumption is that if it is not m2m plan all others are on a contract.
        - payment type - change to autopayment and drop 'credit_card', 'bank_transfer', check' and 'e_check'
        -interent type:
            *change internet type to fiber (Yes or No).
            *Because there was only 61 customers with no internet that churned as validated in the Bivariate stats below.
            *assumption, based on hypothesis, is that a customer is on fiber or not on fiber. 

# Hypothesis Testing

-If our p-value is less than 'alpha' we reject the null hypothesis, otherwise, we fail to reject the null hypothesis.



In [ ]:
alpha = .05

#The p-value is the chance that we obtained the results we did (or would obtain more extreme results) due to chance if the null hypothesis is true.
#If the null hypothesis is true, our t-scores will follow a normal distribution and will be centered around 0.
#The one sample t-test lets us compare the mean for a specific subgroup against the population mean. (One featureb)
#We can also use a t-test to compare the means between two different supgroups. (difference between two features)


### HYP

1. churn is dependent of whether or not customers are on fiber. (chi2)
        - Null: churn is independent of whether or not a customer are on fiber.
        - Alternate: churn is dependent of whether or not customers are on fiber.

In [ ]:

observed = pd.crosstab(train.churned, train['fiber'])
chi2, p, degf, expected = stats.chi2_contingency(observed)
print('Observed\n')
print(observed.values)
print('---\nExpected\n')
print(expected)
print('---\n')
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')
print(f'Is p value less than alpha? {p < alpha}')
if p < alpha:
    print('We reject the null')
else:
    print("we fail to reject the null")


### Hyp

2. tenure and monthly charges are linearly correlated (pearsonr)
        - Null: tenure and monthly charges are not linearly correlated
        - Alt: tenure and monthly charges are linearly correlated

In [ ]:
corr, p = stats.pearsonr(train.tenure, train.monthly_charges)

if p < alpha:
    print("We reject the null hypothesis")
else:
    print("We fail to reject the null hypothesis")
    
corr, p

### Hyp

3. churn is dependent younger customers
        - Null: churn is independent of younger customers.
        - Alternate: churn is dependent of younger customers.

In [ ]:
observed = pd.crosstab(train.churned, train['is_senior'])
chi2, p, degf, expected = stats.chi2_contingency(observed)
print('Observed\n')
print(observed.values)
print('---\nExpected\n')
print(expected)
print('---\n')
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')
print(f'Is p value less than alpha? {p < alpha}')
if p < alpha:
    print('We reject the null')
else:
    print("we fail to reject the null")

### Hyp

4. Do you younger customers pay more then the older customers
        N: The average monthly charges for younger customers is no different than the population (monthly_charges_mean)
        A: The average monthly charges for younger customers are different than rest of the population.

In [ ]:
older_monthly_charges = df[df.is_senior == 0].monthly_charges
monthly_charges_mean = df.monthly_charges.mean()

test_results = stats.ttest_1samp(older_monthly_charges, monthly_charges_mean)
test_results

print(f'Is p value less than alpha? {p < alpha}')
if p < alpha:
    print('We reject the null')
else:
    print("we fail to reject the null")

5. do younger customers churn more than older customers?
        -younger customer churn is no different than the older population (churn)
        -younger customer churn is different than the older population (churn)

In [ ]:
older_churn = df[df.is_senior == 0].churned
churn_mean = df.churned.mean()

test_results = stats.ttest_1samp(older_churn, churn_mean)
test_results

print(f'Is p value less than alpha? {p < alpha}')
if p < alpha:
    print('We reject the null')
else:
    print("we fail to reject the null")

6. Do single customers churn more than non single customers?
        -Single customer churn is no different than the non single population.
        -Single customer churn is different than the non single population.

In [ ]:
single = df[df.not_single == 0].churned
non_single = df.churned.mean()

test_results = stats.ttest_1samp(single, non_single)
test_results

print(f'Is p value less than alpha? {p < alpha}')
if p < alpha:
    print('We reject the null')
else:
    print("we fail to reject the null")

7. churn dependent on autopayment.
        - Null: churn is independent of whether or not customes are enrolle in autopayment plan.
        - Alternate: churn is dependent of whether or not customes are enrolle in autopayment plan.

In [ ]:
observed = pd.crosstab(train.churned, train['autopayment'])
chi2, p, degf, expected = stats.chi2_contingency(observed)
print('Observed\n')
print(observed.values)
print('---\nExpected\n')
print(expected)
print('---\n')
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')
print(f'Is p value less than alpha? {p < alpha}')
if p < alpha:
    print('We reject the null')
else:
    print("we fail to reject the null")

8. churn dependent on phone_service (951)
            - Null: churn is independent on phone services
            - Alternate: churn is dependent on phone services

In [ ]:
observed = pd.crosstab(train.churned, train['phone_service'])
chi2, p, degf, expected = stats.chi2_contingency(observed)
print('Observed\n')
print(observed.values)
print('---\nExpected\n')
print(expected)
print('---\n')
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')
print(f'Is p value less than alpha? {p < alpha}')
if p < alpha:
    print('We reject the null')
else:
    print("we fail to reject the null")

9. Customers who are young, single, on fiber, has phone service, on m2m, and not enrolled in a autopayment plan are more likely to churn than those customers that do not fall in this category.

    -N: Customers who are young, single, on fiber, has phone service, on m2m, and not enrolled in a autopayment plan are no different than the rest of the population
        -A: Customers who are young, single, on fiber, has phone service, on m2m, and not enrolled in a autopayment plan is different than the rest of the population



In [ ]:

customer = train[(train.is_senior == 0) & (train.not_single == 0) & (train.fiber == 1) & (train.m2m == 1) & 
                 (train.phone_service == 1) & (train.autopayment == 0)]
customer_churn = customer.churned
churn = train.churned.mean()

test_results = stats.ttest_1samp(customer_churn, churn)
test_results

print(f'Is p value less than alpha? {p < alpha}')
if p < alpha:
    print('We reject the null')
else:
    print("we fail to reject the null")

#### Bivariate Stats

    1. Plot the interaction of each variable with the target. 
    
    2. Document takeaways.

    3. Explore interation of independent variables using viz and/or hypothesis testing to address interdependence.

In [ ]:
#removed target 'churned'
cat_vars = ['fiber', 'm2m', 'is_senior', 'not_single', 'phone_service','autopayment']
quant_vars = ['tenure', 'monthly_charges']
explore.explore_bivariate(train, 'churned', cat_vars, quant_vars)

##### TAKEAWAYS
    8/28/2021
    
  -ON ALL Trained data
  
     - Number of fiber (728)  customers who churned.
       * what is about our fiber internet service is making customers churn?
       * Less people churn on DSL (257) then on fiber...possible validation that fiber service needs work.
       * DROP no_internet. Only 61 customers who do not have internet churn.
       HYPOTHESIS: Customers on fiber are more likely to churn than those customers not on fiber. ****convert Fiber and DSL into one column - Fiber(Yes or No) and encode.
       *****AND what internet services fiber customers have AND do they affect churn.
       * The new fiber becomes a feature
       * test hyp

    - 90.5% (929/1231) of customers in the first month churn than any other month
        HYPOTHESIS: Customers are more than likely to churn in the first month of service than any other customer.
        
    - senior_citizen:
        HYPOTHESIS: Customers who are NOT senior citizens (765) are more likely to churn than senior citizens (281).
 
    - Between 'partner' and 'dependents' less single (no_d_863-no_p_687=176) customers churn then not single people (p_359 + d_183 = 542).
    
    - Phone service:
        HYPOTHESIS: Customers with phone_service (951) are more likely to churn than those that do not have phone_service?
            ***DROP multiple_lines. It is evenly weighed and does not have an impact on phone_services.
            
    - Interent services: DROP streaming services. Streaming services are eveningly weighed and will not affect intenert services and churn.
        
    - Payment type:
        * converted payment type to autopayment (Yes or No). Assumption is that customers are either on or not on an autopayment plan.can I call you
        HYPOTHESIS: Customers enrolled in automatic payment plan (266) are less likely to churn than those customers NOT enrolled in the automatic payment plan (780).

    - Lower tenure more churn.
    - Higher monthly payments more churn.

  
    
    - Contract type:
        * combine  m2m, 1, and 2 year contracts to a single column titled 'm2m' and encode.
        HYPOTHESIS: m2m churn is greater than those customers not on m2m plan.
        ---------------------------------------------
    QUESTIONS:
        1. Why do customers on fiber churn more?
        2. Why do customers churn more in the first month than any other month?
        3. Why do non single customers (with partner or dependents) churn more then single customers?
        4. Why do younger customers churn more than older customers?
        5. Why do customers on m2m plans churn more than those customers on contracts?
        6. Why do customers enrolled in autopayments churn less than those customers not enrolled in an autopayment?
        7. Why do customers churn more with phone services?
        
        * What do these questions all have in common?
       ** What is the cost per month of a younger customers who is not single, has phone services and fiber, on a m2m plan, enrolled in an autopayment plan?
       
       *** FINAL HYPOTHESIS: Customers who are young, not single, on fiber, has phone service, on m2m, and not enrolled in a autopayment plan are more likely to churn than those customers that do not fall in this category.
        
        MY FEATURES: fiber (YES), tenure (1 month), single (NO), senior citizen (NO), m2m (YES), autopayments (NO), phone_service (YES)
        
        Test and validate HYPOTHESIS(reject or fail to reject final hypothesis)
_________________________________________________


Recommendation:???
    _Churn in one month (213)_
        * Incentive...
            - 1 month free service for m2m customers?
            - Free upgrade service for signing a one_year contract?
            - Free premium service for two year contract?
        * Create a graph that shows churn over tenure.

#### Multivariate Stats

    1. How subgroups compare to each-other and to the overall population. 
    
    2. Answer questions using visualizations and/or hypothesis testing.
    
    (Here I can test some of my questions...reference Exploratory Analysis

    _ Need to drop some columns for the multivariate stats.

In [ ]:
target = 'churned'
explore.explore_multivariate(train, target, cat_vars, quant_vars)

##### TAKEAWAYS

# Train, Validate, Test

In [ ]:
# create X & y version of train, where y is a series with just the target variable and X are all the features.
X_train = train[['fiber', 'm2m', 'is_senior', 'not_single', 'phone_service','autopayment']]#features
y_train = train.churned#target

X_validate = validate[['fiber', 'm2m', 'is_senior', 'not_single', 'phone_service','autopayment']]
y_validate = validate.churned

X_test = test[['fiber', 'm2m', 'is_senior', 'not_single', 'phone_service','autopayment']]
y_test = test.churned

# Establish the baseline

In [ ]:
# The mode is a great baseline
baseline = y_train.mode()

# Produce a boolean array with True representing a match between the baseline prediction and reality
matches_baseline_prediction = y_train == 0

baseline_accuracy = matches_baseline_prediction.mean()
print(f"Baseline accuracy: {round(baseline_accuracy, 2)}")

# Modeling

### Decision Tree

In [ ]:
#create the model

#for classification you can change the algorithm to gini or entropy (information gain).  
#Default is gini.

model = DecisionTreeClassifier(max_depth=4, random_state=123)

In [ ]:
#Fit the model

model = model.fit(X_train, y_train)

#### Make predictions

In [ ]:
#Use the model

# We'll evaluate the model's performance on train, first

y_predictions = model.predict(X_train)

In [ ]:
# Produce the classification report on the actual y values and this model's predicted y values
report = classification_report(y_train, y_predictions, output_dict=True)
print("Tree of 4 depth")
pd.DataFrame(report)

#### Estimate probability

In [ ]:
y_pred_proba = model.predict_proba(X_train)

#### Compute accuracy

In [ ]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(model.score(X_train, y_train)))

#### Confusion matrix

TP
TN
FP
FN

In [ ]:
confusion_matrix(y_train, y_predictions)

In [ ]:
y_train.value_counts()

In [ ]:
labels = sorted(y_train.unique())

pd.DataFrame(confusion_matrix(y_train, y_predictions), index=labels, columns=labels)


In [ ]:
print(classification_report(y_train, y_predictions))

#### Evaluate the Model with our Validate dataset

In [ ]:
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(model.score(X_validate, y_validate)))

In [ ]:
# And since accuracy isn't everything

# Produce y_predictions that come from the X_validate
y_pred = model.predict(X_validate)

# Compare actual y values (from validate) to predicted y_values from the model run on X_validate
print(classification_report(y_validate, y_pred))

In [ ]:
# Produce the classification report on the actual y values and this model's predicted y values
report = classification_report(y_train, y_predictions, output_dict=True)
print("Tree of 1 depth")
pd.DataFrame(report)

### Random Forrest

In [ ]:
#Create the Model
rf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini', min_samples_leaf=1, n_estimators=10,
                            max_depth=4, 
                            random_state=123)

In [ ]:
#Fit the Model
rf.fit(X_train, y_train)

In [ ]:
#Use the Model
#evaluate the weight
print(rf.feature_importances_)

In [ ]:
#estimate
y_pred = rf.predict(X_train)

In [ ]:
#probability
y_pred_proba = rf.predict_proba(X_train)

In [ ]:
#accuracy
print('Accuracy of random forest classifier on training set: {:.2f}'.format(rf.score(X_train, y_train)))

In [ ]:
#confusion matrix
print(confusion_matrix(y_train, y_pred))

In [ ]:
#Create a classificaiton report
print(classification_report(y_train, y_pred))

In [ ]:
print('Accuracy of random forest classifier on test set: {:.2f}'.format(rf.score(X_validate, y_validate)))

### KNN

In [ ]:
#Create the model
# weights = ['uniform', 'density']
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')

In [ ]:
#fit the model
knn.fit(X_train, y_train)

In [ ]:
#predict/use the model
y_pred = knn.predict(X_train)

In [ ]:
#probability
y_pred_proba = knn.predict_proba(X_train)

In [ ]:
#accuracy
print('Accuracy of KNN classifier on training set: {:.2f}'.format(knn.score(X_train, y_train)))

In [ ]:
#confusion matrix
print(confusion_matrix(y_train, y_pred))

In [ ]:
print(classification_report(y_train, y_pred))

In [ ]:
#validate
print('Accuracy of KNN classifier on test set: {:.2f}'.format(knn.score(X_validate, y_validate)))

# Conclusion

    - What we don't know:
        * How plans relate to cost. i.e. phone to service plans
        * How service plans (online security, online backup, tech support, device protection, and streaming services) relates to cost.
        * Cost of one line, cost of multiple lines (what is multiple lines? Is it 2, 3, 4, or more) and how that relates to cost.